In [ ]:
# fashion_pose.py : MPII를 사용한 신체부위 검출
import pandas as pd
import cv2
import os
    
# 각 파일 path
protoFile ="./body_25/pose_deploy.prototxt"
weightsFile ="./body_25/pose_iter_584000.caffemodel"
# 위의 path에 있는 network 불러오기
net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)
    
path ="../squat_x"# 사진 파일 주소
file_list = os.listdir(path) # 사진 파일명 리스트
num_list = [1,5,12,13,14] # 스쿼트
columns_name = ['1x','1y','5x','5y','12x','12y','13x','13y','14x','14y'] # 스쿼트
df = pd.DataFrame(columns=columns_name);
for file in file_list:
    # 이미지 읽어오기
    print(len(df), end=' ')
    print(file)
    image = cv2.imread(path +'/'+file)
    # frame.shape = 불러온 이미지에서 height, width, color 받아옴
    imageHeight, imageWidth, _ = image.shape
    # network에 넣기위해 전처리
    inpBlob = cv2.dnn.blobFromImage(image, 1.0 /255, (imageWidth, imageHeight), (0, 0, 0), swapRB=False, crop=False)
    # network에 넣어주기
    net.setInput(inpBlob)
    # 결과 받아오기
    output = net.forward()
    # output.shape[0] = 이미지 ID, [1] = 출력 맵의 높이, [2] = 너비
    H = output.shape[2]
    W = output.shape[3]
    #print("이미지 ID : ", len(output[0]), ", H : ", output.shape[2], ", W : ",output.shape[3]) # 이미지 ID
    # 키포인트 검출시 이미지에 그려줌
    row = []
    for i in num_list:
        # 해당 신체부위 신뢰도 얻음.
        probMap = output[0, i, :, :]
        # global 최대값 찾기
        minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)
        # 원래 이미지에 맞게 점 위치 변경
        x = (imageWidth * point[0]) / W
        y = (imageHeight * point[1]) / H
        #print(i, " ", x, " ", y);
        
        row.append(x);
        row.append(y);
        # point를 dataframe으로 변경
    df = df.append(pd.Series(row, index=df.columns), ignore_index=True)
    df.to_csv('squat_x.csv', mode='w')
